In [1]:
import pandas as pd
import numpy as np

In [2]:
fname = '/Users/andyliu/Downloads/aapl_2000-2020.csv'
aapl = pd.read_csv(fname)
aapl.head()

,target_value,timestamp,item_id
0,3.7500,2000-01-03 09:30:00,stock
1,3.7478,2000-01-03 09:31:00,stock
2,3.7545,2000-01-03 09:32:00,stock
3,3.7500,2000-01-03 09:33:00,stock
4,3.7344,2000-01-03 09:34:00,stock


In [3]:
fn = lambda x: (x['timestamp'].split(' '))[0]
aapl['date'] = aapl.apply(fn, axis=1)

In [4]:
lify = lambda x: list(x)
aapl = aapl.groupby(['date']).agg(lify)
aapl = aapl['target_value']
aapl.head()

date
2000-01-03    [3.75, 3.7478, 3.7545, 3.75, 3.7344, 3.7299, 3...
2000-01-04    [3.8661, 3.884, 3.8795, 3.9108, 3.8884, 3.8929...
2000-01-05    [3.7032, 3.6965, 3.6965, 3.6965, 3.7032, 3.692...
2000-01-06    [3.7924, 3.8103, 3.7902, 3.817, 3.7991, 3.7991...
2000-01-07    [3.4465, 3.4509, 3.4643, 3.4599, 3.4866, 3.497...
Name: target_value, dtype: object

In [5]:
def four_prices(l):
    '''
    l is in the target_value column
    '''
    open_price = l[0]
    close_price = l[-1]
    l.sort()
    low_price = l[0]
    high_price = l[-1]
    return(open_price, close_price, low_price, high_price)

In [6]:
dates = []
opens = []
closes = []
lows = []
highs = []
indices = list(aapl.index)
aapl_l = list(aapl)
for i in range(0, len(indices)):
    dates.append(indices[i])
    row = four_prices(aapl_l[i])
    opens.append(row[0])
    closes.append(row[1])
    lows.append(row[2])
    highs.append(row[3])

In [7]:
aapl_fourprices = pd.DataFrame(data={'date':dates,'open':opens,'close':closes,'low':lows,'high':highs})

In [8]:
aapl_fourprices['volume'] = np.zeros(len(dates))

In [9]:
aapl_fourprices.head()

,date,open,close,low,high,volume
0,2000-01-03,3.7500,3.9978,3.6384,4.0089,0.0
1,2000-01-04,3.8661,3.6608,3.6295,3.9375,0.0
2,2000-01-05,3.7032,3.7143,3.6786,3.9398,0.0
3,2000-01-06,3.7924,3.3929,3.3929,3.8170,0.0
4,2000-01-07,3.4465,3.5536,3.4152,3.6027,0.0


In [10]:
import ta

In [12]:
df = aapl_fourprices
indicator_bb = ta.volatility.BollingerBands(close=df["close"], n=20, ndev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

In [17]:
indicator_atr = ta.volatility.AverageTrueRange(high=df['high'], low=df['low'], close=df['close'])
df['atr'] = indicator_atr.average_true_range()

In [18]:
indicator_macd = ta.trend.MACD(close=df['close'])
df['macd'] = indicator_macd.macd()
indicator_cci = ta.trend.cci(high=df['high'], low=df['low'], close=df['close'])
df['cci'] = indicator_cci
df.head()

,date,open,close,low,high,volume,bb_bbm,bb_bbh,bb_bbl,atr,macd,cci
0,2000-01-03,3.7500,3.9978,3.6384,4.0089,0.0,NaN,NaN,NaN,0.0,NaN,NaN
1,2000-01-04,3.8661,3.6608,3.6295,3.9375,0.0,NaN,NaN,NaN,0.0,NaN,NaN
2,2000-01-05,3.7032,3.7143,3.6786,3.9398,0.0,NaN,NaN,NaN,0.0,NaN,NaN
3,2000-01-06,3.7924,3.3929,3.3929,3.8170,0.0,NaN,NaN,NaN,0.0,NaN,NaN
4,2000-01-07,3.4465,3.5536,3.4152,3.6027,0.0,NaN,NaN,NaN,0.0,NaN,NaN


In [19]:
indicator_ema = ta.trend.EMAIndicator(close=df['close'])
df['ema'] = indicator_ema.ema_indicator()
df.head()

,date,open,close,low,high,volume,bb_bbm,bb_bbh,bb_bbl,atr,macd,cci,ema
0,2000-01-03,3.7500,3.9978,3.6384,4.0089,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,2000-01-04,3.8661,3.6608,3.6295,3.9375,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,2000-01-05,3.7032,3.7143,3.6786,3.9398,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,2000-01-06,3.7924,3.3929,3.3929,3.8170,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,2000-01-07,3.4465,3.5536,3.4152,3.6027,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [21]:
indicator_roc = ta.momentum.ROCIndicator(close=df['close'])
df['roc'] = indicator_roc.roc()

In [22]:
indicator_sma = ta.trend.sma_indicator(close=df['close'])
df['sma12'] = indicator_sma
indicator_sma = ta.trend.sma_indicator(close=df['close'], n=5)
df['sma5'] = indicator_sma
df.head()

,date,open,close,low,high,volume,bb_bbm,bb_bbh,bb_bbl,atr,macd,cci,ema,roc,sma12,sma5
0,2000-01-03,3.7500,3.9978,3.6384,4.0089,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,3.8661,3.6608,3.6295,3.9375,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,3.7032,3.7143,3.6786,3.9398,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,3.7924,3.3929,3.3929,3.8170,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,3.4465,3.5536,3.4152,3.6027,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.66388


In [24]:
stochasticoscillator = ta.momentum.StochasticOscillator(high=df['high'], low=df['low'], close=df['close'])
df['stochastic_oscillator'] = stochasticoscillator.stoch()
df.head()

,date,open,close,low,high,volume,bb_bbm,bb_bbh,bb_bbl,atr,macd,cci,ema,roc,sma12,sma5,stochastic_oscillator
0,2000-01-03,3.7500,3.9978,3.6384,4.0089,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,3.8661,3.6608,3.6295,3.9375,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,3.7032,3.7143,3.6786,3.9398,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,3.7924,3.3929,3.3929,3.8170,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,3.4465,3.5536,3.4152,3.6027,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.66388,NaN


In [25]:
mtm6 = ta.momentum.roc(close=df['close'], n=6)
df['mtm6'] = mtm6
mtm12 = ta.momentum.roc(close=df['close'], n=12)
df['mtm12'] = mtm12
df.head()

,date,open,close,low,high,volume,bb_bbm,bb_bbh,bb_bbl,atr,macd,cci,ema,roc,sma12,sma5,stochastic_oscillator,mtm6,mtm12
0,2000-01-03,3.7500,3.9978,3.6384,4.0089,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,3.8661,3.6608,3.6295,3.9375,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,3.7032,3.7143,3.6786,3.9398,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,3.7924,3.3929,3.3929,3.8170,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,3.4465,3.5536,3.4152,3.6027,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3.66388,NaN,NaN,NaN


In [26]:
df.to_csv('./aapl_ta.csv')

In [34]:
df3 = df.iloc[25:]
df3.head()
df3.to_csv('./aapl_indicators.csv')